In [105]:
import numpy as np
import matplotlib.pyplot as plt
data = np.load('/home/somya/SML_2/mnist.npz')
for file in data.files:
  print(file)

x_test
x_train
y_train
y_test


In [153]:
X_test=np.zeros((3147,784))
Y_test = np.zeros((3147,))
data_test = data['x_test'].reshape(10000,-1)
print(data_test.shape)
for k in range(3):
  j=0
  indices = np.where(data['y_test']==k)[0]
  for i in indices:
    X_test[j] = data_test[i]
    Y_test[j] = k
    j+=1
  print(len(indices))
X_test = X_test.T
cov_matrix = np.dot(X_test,X_test.T)/18622
eigenvalues , eigenvectors = np.linalg.eigh(cov_matrix)
sorted = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted]
eigenvectors = eigenvectors[:, sorted]
U = eigenvectors
print(U.shape)
X_test=np.dot(X_test.T, U[:, :10])
print(X_test.shape)
print(Y_test.shape)

(10000, 784)
980
1135
1032
(784, 784)
(3147, 10)
(3147,)


In [130]:
X = np.zeros((18623,784))
Y = np.zeros(18623)
dataset = data['x_train'].reshape(60000,-1)
for k in range(3):
  j=0
  indices = np.where(data['y_train']==k)[0]
  for i in indices:
    X[j] = dataset[i]
    Y[j] = k
    j+=1
  print(len(indices))
X=X.T


5923
6742
5958


In [131]:
print(Y[10])
# Y= Y.reshape(-1,1)

2.0


In [133]:
print(X.shape)
means = np.mean(X,axis=1,keepdims=True)
X = X - means

(784, 18623)


In [134]:
cov_matrix = np.dot(X,X.T)/18622
print(Y[0])

2.0


In [135]:
print(cov_matrix.shape)
eigenvalues , eigenvectors = np.linalg.eigh(cov_matrix)
sorted = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted]
eigenvectors = eigenvectors[:, sorted]
U = eigenvectors
print(U.shape)
X=np.dot(X.T, U[:, :10])

(784, 784)
(784, 784)


In [136]:
print(type(X))
print(X.shape)
print(Y.shape)

<class 'numpy.ndarray'>
(18623, 10)
(18623,)


In [137]:
import pandas as pd
df = pd.DataFrame(X,columns=['column0','column1','column2','column3','column4','column5','column6','column7','column8','column9'])
print(df.shape)
print(Y.shape)
# k = Y.tolist()
# print(type(k))
# df['Target_value'] = k

(18623, 10)
(18623,)


In [138]:
medians=[]
for i in range(10):
     k = df.iloc[:, i].median()
     medians.append(k)

In [85]:
# Y = Y.tolist()
# print(Y[0])

In [139]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [232]:
from collections import Counter
def get_most_occurring_feature(classes):
        counter = Counter(classes)
        k, v = counter.most_common(1)[0]
        return k
def partition_classes(X,y,attribute,val):
    X_left=[]
    X_right=[]

    y_left=[]
    y_right=[]
    for i in range(len(X)):
        if float(X[i][attribute]<=val):
            X_left.append(X[i])
            y_left.append(y[i])
        else:
            X_right.append(X[i])
            y_right.append(y[i])
    return X_left,X_right,y_left,y_right
class DecisionNode:
    def __init__(self,left,right,decison_function,class_label=None):
        self.left = left
        self.right = right
        self.decision_function = decison_function
        self.class_label = class_label
    def decide(self,feature):
        if self.class_label is not None:
            return self.class_label
        elif self.decision_function(feature):
            return self.left.decide(feature)
        else:
            return self.right.decide(feature)
def gini_impurity(class_vector):
        counts = Counter(class_vector)
        prob_zero = counts[0]/len(class_vector)
        prob_one = counts[1]/len(class_vector)
        prob_two = counts[2]/len(class_vector)
        prob_sum = prob_zero**2 + prob_one**2 + prob_two**2
        return 1-prob_sum
def gini_gain(pre_classes,curr_classes):
    pre_gini_gain = gini_impurity(pre_classes)
    curr_gini_gain=0
    pre_len = len(pre_classes)
    if len(curr_classes[0])==0 or len(curr_classes[1])==0:
        return 0
    for ll in curr_classes:
        curr_length = len(ll)
        curr_gini_gain += gini_impurity(ll) * float(curr_length)/pre_len
    return pre_gini_gain-curr_gini_gain

class DecisionTree:
    def __init__(self,features,classes,depth_limit=2):
        self.root = None
        self.depth_limit = depth_limit
        self.features = features
        self.classes = classes
    
    def __build_tree__(self,features,classes,depth=0):
        best_gain =-1
        best_column =-1
        best_threshold=-1
        if(len(classes)==0):
            return None
        elif len(classes)==1:
            return DecisionNode(None,None,None,classes[0])
        elif np.all(classes[0]==classes[:]):
            return DecisionNode(None,None,None,classes[0])
        elif depth == self.depth_limit:
            return DecisionNode(None,None,None,get_most_occurring_feature(classes))
        else:
            for i in range(features.shape[1]):
                values = features[:,i]
                column_mean = np.mean(values)
                classes_new =[]
                temp_X_left,temp_X_right,temp_y_left,temp_y_right = partition_classes(features,classes,i,column_mean)
                classes_new.append(temp_y_left)
                classes_new.append(temp_y_right)
                local_gini = gini_gain(classes,classes_new)
                if local_gini > best_gain:
                    best_gain = local_gini
                    best_column = i
                    best_threshold = column_mean
            X_left ,X_right,y_left,y_right = partition_classes(features,classes,best_column,best_threshold)
            depth+=1
            left_tree = self.__build_tree__(np.array(X_left),np.array(y_left),depth)
            right_tree = self.__build_tree__(np.array(X_right),np.array(y_right),depth)
            return DecisionNode(left_tree,right_tree,lambda feature: feature[best_column] < best_threshold)
    def fit(self,features,classes):
        self.root = self.__build_tree__(features,classes)
    def classify1(self,features):
        tree = self.root
        return tree.decide(features)
    def classify(self,features):
        class_labels =[]
        for feature in features:
            tree = self.root
            class_labels.append(tree.decide(feature))
        return class_labels


In [233]:
tree = DecisionTree(df,Y,2)
tree.fit(X,Y)

In [234]:
print(type(X_test))

<class 'numpy.ndarray'>


In [235]:
labels = tree.classify(X_test)

In [236]:
print(type(labels))
print(type(Y_test))
print(len(labels))
print(len(Y_test))
j=0
count=0
for i in labels :
    if(i==Y_test[j]):
        count+=1
print(count)
print(count/len(labels)*100)
print(X.shape)

<class 'list'>
<class 'numpy.ndarray'>
3147
3147
2012
63.93390530664125
(18623, 10)


In [237]:
#question 4
def find_mode(arr):
     frequency_dict = {}
     for item in arr:
        if item in frequency_dict:
            frequency_dict[item] += 1
        else:
            frequency_dict[item] = 1
     max_frequency = max(frequency_dict.values())
     modes = [key for key, value in frequency_dict.items() if value == max_frequency]
     return modes[0]

class Bagging:
    def __init__(self,n_estimators=5):
        self.n_estimators = n_estimators
        self.models=[]
    def fit(self,X,y):
        samples = X.shape[0]
        for i in range(self.n_estimators):
            idx = np.random.choice(samples,samples,replace=True)
            X_bootstrap = X[idx]
            y_bootstrap = y[idx]
           
            model = DecisionTree(X_bootstrap,y_bootstrap)
            model.fit(X_bootstrap,y_bootstrap)
            self.models.append(model)
    def predict(self,X):
        predictions = np.array([model.classify1(X) for model in self.models])
        prediction = find_mode(predictions)
        return prediction


In [238]:
bag = Bagging(5)
bag.fit(X,Y)

In [241]:
label=[]
for i in X_test:
    label.append(bag.predict(i))


In [242]:
j=0
count1=0
for i in labels :
    if(i==Y_test[j]):
        count1+=1
    j+=1
print(count1)

4
